# Notebook to plot the distribution of the caudal-rostral length of the spinal levels

In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os

In [20]:
# Path to csv files with information about the spinal levels
PATH_pred = "/Users/theomathieu/Documents/Stage/results_img/spine-generic/dist_pred.csv"

In [21]:
df_pred = pd.read_csv(PATH_pred)
display(df_pred.head())

,level,sub_name,spinal_start,spinal_end,height,vertebrae_start,vertebrae_end
0,2,sub-mountSinai02_103_0000,208.0,219.0,8.8,0,1
1,3,sub-mountSinai02_103_0000,184.0,203.0,15.2,0,1
2,4,sub-mountSinai02_103_0000,166.0,179.0,10.4,0,1
3,5,sub-mountSinai02_103_0000,144.0,155.0,8.8,0,1
4,6,sub-mountSinai02_103_0000,123.0,136.0,10.4,0,1


In [22]:
# Add z-score to find outliers
z_scores = (df_pred['height'] - df_pred['height'].mean()) / df_pred['height'].std()
df_pred["z_scores"] = z_scores

### Dataframe creation

In [23]:
df_height = df_pred[["height", "level"]].copy()
df_height["mesure"] = "length"
df_height["from"] = "pred_value"
df_height.rename(columns={"height": "value"}, inplace=True)
display(df_height.head())

,value,level,mesure,from
0,8.8,2,length,pred_value
1,15.2,3,length,pred_value
2,10.4,4,length,pred_value
3,8.8,5,length,pred_value
4,10.4,6,length,pred_value


In [24]:
for level in df_height["level"].unique():
    new_line_mean = pd.DataFrame({"value": [df_height.loc[df_height["level"] == level, "value"].mean()],
                                  "level": [level.astype(int)],
                                  "mesure": ["mean"],
                                  "from": ["pred"]})
    df_height = pd.concat([df_height, new_line_mean], ignore_index=True)
    new_line_std = pd.DataFrame({"value": [df_height.loc[df_height["level"] == level, "value"].std()],
                                 "level": [level.astype(int)],
                                 "mesure": ["std"],
                                 "from": ["pred"]})
    df_height = pd.concat([df_height, new_line_std], ignore_index=True)
display(df_height.head())

,value,level,mesure,from
0,8.8,2,length,pred_value
1,15.2,3,length,pred_value
2,10.4,4,length,pred_value
3,8.8,5,length,pred_value
4,10.4,6,length,pred_value


In [25]:
cadotte = {
    '3': {'mean': 10.5, 'std': 2.2},
    '4': {'mean': 9.9, 'std': 1.3},
    '5': {'mean': 10.5, 'std': 1.5},
    '6': {'mean': 9.7, 'std': 1.6},
    '7': {'mean': 9.4, 'std': 1.4},
    '8': {'mean': 9.6, 'std': 1.4},
}
df_cadotte_mean = pd.DataFrame(cadotte).T[['mean']]
df_cadotte_mean["level"] = df_cadotte_mean.index.astype(int)
df_cadotte_mean["from"] = "cadotte"
df_cadotte_mean["mesure"] = "mean"
df_cadotte_mean.rename(columns={"mean": "value"}, inplace=True)
df_cadotte_std = pd.DataFrame(cadotte).T[['std']]
df_cadotte_std["level"] = df_cadotte_mean.index.astype(int)
df_cadotte_std["from"] = "cadotte"
df_cadotte_std["mesure"] = "std"
df_cadotte_std.rename(columns={"std": "value"}, inplace=True)
df_cadotte = pd.concat([df_cadotte_mean, df_cadotte_std], ignore_index=True)
display(df_cadotte.head())
df = pd.concat([df_height, df_cadotte], ignore_index=True)
display(df.head())
df_mean_std = df[df["mesure"] != "length"]
pivot_df = df_mean_std.pivot_table(index=['level', 'from'], columns='mesure', values='value', aggfunc='first')

# Reset the index
pivot_df = pivot_df.reset_index()
pivot_df['level_offset'] = pivot_df.apply(
    lambda row: row['level'] - 0.1 if row['from'] == 'pred' else row['level'] + 0.1, axis=1)
display(pivot_df.head())

,value,level,from,mesure
0,10.5,3,cadotte,mean
1,9.9,4,cadotte,mean
2,10.5,5,cadotte,mean
3,9.7,6,cadotte,mean
4,9.4,7,cadotte,mean


,value,level,mesure,from
0,8.8,2,length,pred_value
1,15.2,3,length,pred_value
2,10.4,4,length,pred_value
3,8.8,5,length,pred_value
4,10.4,6,length,pred_value


mesure,level,from,mean,std,level_offset
0,2,pred,5.430159,2.128017,1.9
1,3,cadotte,10.500000,2.200000,3.1
2,3,pred,10.422222,2.242638,2.9
3,4,cadotte,9.900000,1.300000,4.1
4,4,pred,6.953448,2.296077,3.9


In [26]:
fig = px.scatter(pivot_df, x="level_offset", y="mean", color="from", error_y="std")
fig.update_layout(
    title="Caudal-rostral length of the spinal levels",
    xaxis_title="Spinal level",
    yaxis_title="Caudal-rostral length (mm)",
    xaxis=dict(ticktext=["C" + x.astype(str) for x in df["level"].unique()], tickvals=df["level"].unique())
)
fig.show()

In [27]:
fig = px.scatter(pivot_df, x="level_offset", y="mean", color="from", error_y="std")
for level in df["level"].unique():
    df_level = df[df["level"] == level]
    mean = df_level.loc[df_level["mesure"] == "mean"]
    std = df_level.loc[df_level["mesure"] == "std"]
    #TODO handle nan values
    scatter = px.scatter(df_level, x="level", y="value").data[0]
    fig.add_trace(scatter)

fig.show()

## PMJ DISTANCE

In [31]:
PATH_pmj = "/Users/theomathieu/Documents/Stage/results_img/cadotte/csv_file_pred/sub-1-13696_006_0000_pmj-dist.csv"
df_pmj = pd.read_csv(PATH_pmj)
display(df_pmj.head())
PATH_cad = "/Users/theomathieu/Documents/Stage/results_img/cadotte/root.csv"
df_full_cad = pd.read_csv(PATH_cad)
df_cad = df_full_cad[df_full_cad["sub_name"]=="sub-1-13696_006"]
display(df_cad.head())

,level,sub_name,spinal_start,spinal_end,height,PMJ_start,PMJ_end
0,2,sub-1-13696_006_0000,389.0,384.0,1.953000,-40.521296,-42.576228
1,3,sub-1-13696_006_0000,356.0,328.0,10.936798,-54.124618,-66.514692
2,4,sub-1-13696_006_0000,316.0,316.0,0.000000,-71.609619,-71.609619
3,5,sub-1-13696_006_0000,NaN,NaN,NaN,NaN,NaN
4,6,sub-1-13696_006_0000,NaN,NaN,NaN,NaN,NaN


,sub_name,level,PMJ_start,PMJ_end,slice_s,slice_e
84,sub-1-13696_006,9,-148.491625,-162.223213,126,95
85,sub-1-13696_006,8,-135.669504,-143.498630,157,138
86,sub-1-13696_006,7,-118.958511,-129.029305,199,174
87,sub-1-13696_006,6,-102.145586,-112.318312,241,216
88,sub-1-13696_006,5,-86.996992,-97.848986,279,252


In [33]:
subject = 1
colors = ['#5B6C5D', '#5CC878', '#FF707D', '#64DEDC', '#D682C6', '#E1E074','#738282', '#C16200', '#197278', '#EF9CDA']
fig = go.Figure()
print(f"Level\tstart(pred/true)\tend(pred/true)")
for idx, level in enumerate(df_pmj["level"].unique()):
    df_level_pmj = df_pmj[df_pmj["level"] == level]
    df_level_cad = df_cad[df_cad["level"] == level]
    if not np.isnan(df_level_pmj["PMJ_start"].values[0]):
        pred_s = df_level_pmj["PMJ_start"].values[0]
        pred_e = df_level_pmj["PMJ_end"].values[0]
        fig.add_shape(
            type="rect",
            x0=subject-0.3, y0=pred_s, x1=subject-0.05, y1=pred_e,
            line=dict(color=colors[idx], width=2),
            fillcolor=colors[idx],
            name=f"Predicted {level}",
        )
    else:
        pred_s = np.nan
        pred_e = np.nan
    try:
        if not np.isnan(df_level_cad["PMJ_start"].values[0]):
            true_s = df_level_cad["PMJ_start"].values[0]
            true_e = df_level_cad["PMJ_end"].values[0]
            fig.add_shape(
                type="rect",
                x0=subject+0.05, y0=true_s, x1=subject+0.3, y1=true_e,
                line=dict(color=colors[idx], width=3),
                name=f"Cadotte {level}"
            )
        else:
            true_s = np.nan
            true_e = np.nan
    except:
        true_s = np.nan
        true_e = np.nan
        pass
    print(f"{level}\t{pred_s:.2f}/{true_s:.2f}\t{pred_e:.2f}/{true_e:.2f}")
fig.update_layout(
    title="Nerve rootlet layout",
    xaxis_title="Subject",
    yaxis_title="PMJ distance (PMJ = 0 mm)",
    xaxis_range=[0, 10],  # Adjust the x-axis range as needed
    yaxis_range=[min(min(df_pmj["PMJ_end"]), min(df_cad["PMJ_end"]))-25, 2],  # Adjust the y-axis range as needed
    yaxis_autorange='reversed',
    showlegend=True,
    xaxis=dict(ticktext=[str(x) for x in range(1,10)], tickvals=[*range(1,10)]),
    yaxis=dict(dtick=10)


)
fig.show()

Level	start(pred/true)	end(pred/true)
2	-40.52/nan	-42.58/nan
3	-54.12/-50.69	-66.51/-64.17
4	-71.61/-69.16	-71.61/-78.10
5	nan/-87.00	nan/-97.85
6	nan/-102.15	nan/-112.32
7	-121.30/-118.96	-130.59/-129.03
8	nan/-135.67	nan/-143.50
9	nan/-148.49	nan/-162.22
10	nan/nan	nan/nan
11	nan/nan	nan/nan


In [34]:
df_root_pd = pd.read_csv("/Users/theomathieu/Documents/Stage/results_img/cadotte/root.csv")
df_ver_pd = pd.read_csv("/Users/theomathieu/Documents/Stage/results_img/cadotte/vertebrae.csv")
path_pred =  "/Users/theomathieu/Documents/Stage/results_img/cadotte/csv_file_pred/"
display(df_root_pd.head())
display(df_ver_pd.head())

,sub_name,level,PMJ_start,PMJ_end,slice_s,slice_e
0,sub-5-13755_009,9,-136.924703,-145.959067,134,114
1,sub-5-13755_009,8,-122.710674,-132.220648,167,145
2,sub-5-13755_009,7,-111.824036,-120.367074,193,173
3,sub-5-13755_009,6,-98.628645,-106.831245,226,205
4,sub-5-13755_009,5,-83.462261,-93.617861,264,238


,sub_name,level,PMJ_start,PMJ_end,slice_s,slice_e
0,sub-5-13755_009,1,-5.485255,-67.354024,454,303
1,sub-5-13755_009,2,-69.799487,-85.805861,297,258
2,sub-5-13755_009,3,-96.904453,-116.426227,230,182
3,sub-5-13755_009,4,-126.922265,-132.220648,157,145
4,sub-11-14411_013,1,-9.782092,-56.112973,450,336


In [38]:
subject = 1
#colors = ['#5CC878', '#FF707D', '#64DEDC', '#D682C6', '#E1E074', '#738282', '#C16200', '#197278', '#EF9CDA']
colors = {2: '#5B6C5D', 3: '#5CC878', 4: '#FF707D', 5: '#64DEDC', 6: '#D682C6', 7: '#E1E074', 8: '#738282', 9: '#C16200', 10: '#197278', 11: '#EF9CDA'}

fig = go.Figure()
for i, subject in enumerate(df_root_pd["sub_name"].unique()):
    df_root_sub = df_root_pd[df_root_pd["sub_name"] == subject]
    df_ver_sub = df_ver_pd[df_ver_pd["sub_name"] == subject]
    id = int(subject.split("-")[1])
    for idx, level in enumerate(df_root_sub["level"].unique()):
        df_level = df_root_sub[df_root_sub["level"] == level]
        if not np.isnan(df_level["PMJ_start"].values[0]):
            root_s = df_level["PMJ_start"].values[0]
            root_e = df_level["PMJ_end"].values[0]
            fig.add_shape(
                type="rect",
                x0=id - 0.175, y0=root_e, x1=id - 0.425, y1=root_s,
                line=dict(color=colors[level], width=2),
                #fillcolor=colors[level],
                name=f"Predicted {level}",
            )
        else:
            root_s = np.nan
            root_e = np.nan
    for idx, level in enumerate(df_ver_sub["level"].unique()[1:]):
        df_level = df_ver_sub[df_ver_sub["level"] == level]
        if not np.isnan(df_level["PMJ_start"].values[0]):
            ver_s = df_level["PMJ_start"].values[0]
            ver_e = df_level["PMJ_end"].values[0]
            fig.add_shape(
                type="rect",
                x0=id + 0.125, y0=ver_s, x1=id - 0.125, y1=ver_e,
                line=dict(color="grey", width=1),
                fillcolor="#F4F4CC",
                name=f"Cadotte {level}"
            )
        else:
            ver_s = np.nan
            ver_e = np.nan
    df_pred_sub = pd.read_csv(os.path.join(path_pred , subject + "_0000_pmj-dist.csv"))
    for idx, level in enumerate(df_pred_sub["level"].unique()[1:]):
        df_level = df_pred_sub[df_pred_sub["level"] == level]
        try:
            if not np.isnan(df_level["PMJ_start"].values[0]):
                pred_s = df_level["PMJ_start"].values[0]
                pred_e = df_level["PMJ_end"].values[0]
                fig.add_shape(
                    type="rect",
                    x0=id + 0.175, y0=pred_e, x1=id + 0.425, y1=pred_s,
                    line=dict(color=colors[level], width=2),
                    fillcolor=colors[level],
                    name=f"Predicted {level}",
                )
            else:
                pred_s = np.nan
                pred_e = np.nan
        except:
            pred_s = np.nan
            pred_e = np.nan
            pass



fig.update_layout(
    title="Nerve rootlet layout",
    xaxis_title="Subject",
    yaxis_title="PMJ distance (PMJ = 0 mm)",
    xaxis_range=[0, 15],  # Adjust the x-axis range as needed
    yaxis_range=[min(df_root_pd["PMJ_end"]) - 5, max(df_root_pd["PMJ_start"])+5],  # Adjust the y-axis range as needed
    yaxis_autorange='reversed',
    showlegend=True,
    xaxis=dict(ticktext=[str(x) for x in range(1, 15)], tickvals=[*range(1, 15)]),
    yaxis=dict(dtick=10)
)
fig.show()